In [2]:
import math

In [3]:
def forward_geodetic_task(phi1, lambda1, s, alpha12, a, b):
    f = (a - b) / a
    # parametric latitude of point P1
    beta1 = math.atan((1 - f) * math.tan(phi1))
    # angular distance on a sphere from the equator to point P1
    sigma1 = math.atan((math.tan(beta1) / math.cos(alpha12)))
    # α azimuth of a geodetic line on the equator
    sin_alpha = math.cos(beta1) * math.sin(alpha12)
    cos2_alpha = 1 - sin_alpha ** 2
    #  geodetic line constant u2
    u2 = cos2_alpha * ((a ** 2 - b ** 2) / (b ** 2))
    # Vincenty constant A
    A = 1 + (u2 / 16384) * (4096 + u2 * (-768 + u2 * (320 - 175 * u2)))
    # Vincenty constant B
    B = (u2 / 1024) * (256 + u2 * (-128 + u2 * (74 - 47 * u2)))
    sigma = s / (b * A)
    
    while True:
        # σm angular distance on the sphere from the equator to the middle of the geodetic line
        sigma2m = 2 * sigma1 + sigma
        delta_sigma = B * math.sin(sigma) * (math.cos(sigma2m) + (B / 4) * ((math.cos(sigma) * (-1 + 2 * (math.cos(sigma2m))**2)) - (B / 6) * math.cos(sigma2m) * (-3 + 4 * math.sin(sigma) ** 2) * (-3 + 4 * math.cos(sigma2m)**2)))
        new_sigma = s / (b * A) + delta_sigma

        if abs(new_sigma - sigma) < 1e-12:  
            break

        sigma = new_sigma

    phi2 = math.atan2((math.sin(beta1) * math.cos(sigma) + math.cos(beta1) * math.sin(sigma) * math.cos(alpha12)),((1 - f) * math.sqrt(sin_alpha ** 2 + (math.sin(beta1) * math.sin(sigma) - math.cos(beta1) * math.cos(sigma) * math.cos(alpha12)) ** 2)))
    lambda0 = math.atan2(math.sin(sigma) * math.sin(alpha12), (math.cos(beta1) * math.cos(sigma) - math.sin(beta1) * math.sin(sigma) * math.cos(alpha12)))
    C = (f / 16) * cos2_alpha * (4 + f * (4 - 3 * cos2_alpha))
    L = lambda0 - (1 - C) * f * sin_alpha * (sigma + C * math.sin(sigma) * (math.cos(sigma2m) + C * math.cos(sigma) * (-1 + 2 * (math.cos(sigma2m)**2))))
    lambda2 = L + lambda1
    alpha21 = math.atan2(sin_alpha, (-math.sin(beta1) * math.sin(sigma) + math.cos(beta1) * math.cos(sigma) * math.cos(alpha12)))

    return phi2, lambda2, alpha21


phi1 = math.radians(0)  
lambda1 = math.radians(0)  
s = 100000  
alpha12 = math.radians(45)  
a = 6378137  
b = 6356752.314245  

phi2, lambda2, alpha21 = forward_geodetic_task(phi1, lambda1, s, alpha12, a, b)

phi2_deg = math.degrees(phi2)
lambda2_deg = math.degrees(lambda2)
alpha21_deg = math.degrees(alpha21)

print("Geodetic coordinates of the second point:")
print(f"ϕ2: {phi2_deg} degrees")
print(f"λ2: {lambda2_deg} degrees")
print(f"α21: {alpha21_deg} degrees")

Geodetic coordinates of the second point:
ϕ2: 0.6394723347288623 degrees
λ2: 0.6352310291253256 degrees
α21: 45.0035449478708 degrees


In [7]:
def inverse_geodetic_problem(phi1, lambda1, phi2, lambda2, a, b):
    f = (a - b) / a
    #  parametric latitude of first point
    beta1 = math.atan((1 - f) * math.tan(phi1))
    # parametric latitude of second point
    beta2 = math.atan((1 - f) * math.tan(phi2))
    # difference in longitude
    L = abs(lambda2 - lambda1)
    lambda_ = L
    
    while True:
        sin_sigma = math.sqrt((math.cos(beta2) * math.sin(lambda_))**2 + 
                              (math.cos(beta1) * math.sin(beta2) * math.cos(lambda_))**2)
        cos_sigma = math.sin(beta1) * math.sin(beta2) + math.cos(beta1) * math.cos(beta2) * math.cos(lambda_)
        # angular distance on sphere between points P1 and P2
        sigma = math.atan2(sin_sigma, cos_sigma)
        sin_alpha = (math.cos(beta1) * math.cos(beta2) * math.sin(lambda_)) / sin_sigma
        cos_alpha2 = 1 - sin_alpha**2
        cos_2sigmam = cos_sigma - (2 * math.sin(beta1) * math.sin(beta2) / cos_alpha2)
        C = f / 16 * cos_alpha2 * (4 + f * (4 - 3 * cos_alpha2))
        lambda_i = L + (1 - C) * f * sin_alpha * (sigma + C * sin_sigma * (cos_2sigmam + C * cos_sigma * (-1 + 2 * cos_2sigmam**2)))
        
        if abs(lambda_i - lambda_) <= 1e-12:
            break
        
        lambda_ = lambda_i
    
    u_sq = cos_alpha2 * ((a**2 - b**2) / b**2)
    A = 1 + u_sq / 16384 * (4096 + u_sq * (-768 + u_sq * (320 - 175 * u_sq)))
    B = u_sq / 1024 * (256 + u_sq * (-128 + u_sq * (74 - 47 * u_sq)))
    delta_sigma = B * sin_sigma * (cos_2sigmam + (1/4) * B * (cos_sigma * (-1 + 2 * cos_2sigmam**2) - (1/6) * B * cos_2sigmam * (-3 + 4 * sin_sigma**2) * (-3 + 4 * cos_2sigmam**2)))
    s = b * A * (sigma - delta_sigma)
    
    alpha12 = math.atan2((math.cos(beta2) * math.sin(lambda_)), (math.cos(beta1) * math.sin(beta2) - math.sin(beta1) * math.cos(beta2) * math.cos(lambda_)))
    alpha21 = math.atan2((math.cos(beta1) * math.sin(lambda_)), (-math.sin(beta1) * math.cos(beta2) + math.cos(beta1) * math.sin(beta2) * math.cos(lambda_)))
    
    return s, alpha12, alpha21


phi1 = math.radians(52.5200)  
lambda1 = math.radians(13.4050)  
phi2 = math.radians(48.8566)  
lambda2 = math.radians(2.3522)  
a = 6378137.0  
b = 6356752.3  

s, alpha12, alpha21 = inverse_geodetic_problem(phi1, lambda1, phi2, lambda2, a, b)


print("Length of the geodetic line (s):", s)
print("Azimuth from point 1 to point 2 (alpha12):", math.degrees(alpha12))
print("Azimuth from point 2 to point 1 (alpha21):", math.degrees(alpha21))


Length of the geodetic line (s): 2808771.751172601
Azimuth from point 1 to point 2 (alpha12): 113.19545162586098
Azimuth from point 2 to point 1 (alpha21): 121.76147972287349
